In [5]:
# !pip install transformers torch

from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from google.colab import drive
import torch

# ==========================================
# 1. 모델 로드 (구글 드라이브)
# ==========================================
drive.mount('/content/drive')
model_path = "/content/drive/MyDrive/toefl_model"

print("⏳ 모델 로딩 중...")

try:
    # 1. 모델과 토크나이저를 직접 로드 (안전장치)
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    # [핵심] function_to_apply="none" 추가!
    scorer = pipeline("text-classification", model=model, tokenizer=tokenizer, function_to_apply="none")
    print("✅ 모델 준비 완료! (Regression Mode On)")

except Exception as e:
    print(f"❌ 모델 로드 중 경고/에러 발생: {e}")
    print("⚠️ 기본 모델로 테스트를 진행합니다.")
    model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-small", num_labels=1)
    tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-small")
    scorer = pipeline("text-classification", model=model, tokenizer=tokenizer, function_to_apply="none")


def predict_score(essay_text):
    """에세이를 입력받아 점수를 반환하는 함수 (스케일 보정 포함)"""
    # 1. 모델 예측 (512 토큰 제한)
    result = scorer(essay_text, truncation=True, max_length=512)
    raw_score = result[0]['score']

    # 2. 점수 보정 (Calibration)
    final_score = raw_score * 3.5

    # 3. 최대 30점, 최소 0점으로 범위 제한
    final_score = min(30.0, max(0.0, final_score))

    return final_score

# ==========================================
# 2. 시연용 에세이 예시 (Demo Cases)
# ==========================================

# Case 1: High Score (30점 만점 예상)
essay_high = """
The lecturer argues that the plan to use bacteria to clean up the oil spill is not effective.
First, the reading claims that the bacteria can break down the oil quickly. However, the lecturer points out that this process is only effective in warm waters. Since the spill occurred in a cold region, the bacteria would multiply too slowly to make a significant difference.
Second, while the reading suggests that the bacteria are safe for the ecosystem, the lecturer refutes this by mentioning that the large number of bacteria could disrupt the local food chain.
This clearly demonstrates that the proposed solution has significant flaws when applied to real-world scenarios. The logic provided by the lecturer effectively counters the reading passage's optimistic view.
"""

# Case 2: Medium Score (24점 예상)
essay_medium = """
The reading says bacteria is good for oil spill. But lecturer disagree.
First reading say it is fast. But lecture say it is cold so it is slow. The bacteria need warm water to eat oil.
Second, reading say it is safe. But lecture say no. Too many bacteria is bad for fish.
So the plan is not good and we need to think about other ways.
"""

# Case 3: Low Score (20점 이하 예상)
essay_low = """
Oil spill is very bad. I think we need to clean it. Bacteria is small.
I don't know if it works. Computers are good.
"""

# ==========================================
# 3. 채점 실행 및 결과 출력
# ==========================================
if 'scorer' in locals():
    print("\n=== 📝 TOEFL AI Scorer Demo ===\n")

    s1 = predict_score(essay_high)
    print(f"[Case 1: High Quality Essay]")
    print(f"▶ Predicted Score: {s1:.2f} / 30.00\n")

    s2 = predict_score(essay_medium)
    print(f"[Case 2: Medium Quality Essay]")
    print(f"▶ Predicted Score: {s2:.2f} / 30.00\n")

    s3 = predict_score(essay_low)
    print(f"[Case 3: Low Quality Essay]")
    print(f"▶ Predicted Score: {s3:.2f} / 30.00\n")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
⏳ 모델 로딩 중...


The tokenizer you are loading from '/content/drive/MyDrive/toefl_model' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e.  This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
Device set to use cuda:0


✅ 모델 준비 완료! (Regression Mode On)

=== 📝 TOEFL AI Scorer Demo ===

[Case 1: High Quality Essay]
▶ Predicted Score: 27.80 / 30.00

[Case 2: Medium Quality Essay]
▶ Predicted Score: 27.31 / 30.00

[Case 3: Low Quality Essay]
▶ Predicted Score: 17.96 / 30.00

